In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FB3AL2VX6 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-147535
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-147535


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster-demo"

if cpu_cluster_name in ws.compute_targets:
    cpu_cluster = ws.compute_targets[cpu_cluster_name]
    print("Using existing cluster: {}".format(cpu_cluster_name))
else:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    print("Creating new compute cluster")

cpu_cluster.wait_for_completion(show_output=True)


Creating new compute cluster
Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        "--C": uniform(0,1),
        "max_iter": choice(50, 100)
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [16]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
#hyperdrive_run.wait_for_completion(show_output=True)
#assert(hyperdrive_run.get_status() == 'Completed')
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [26]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_model = hyperdrive_run.get_best_run_by_primary_metric()
best_model_metrics = best_model.get_metrics()
best_model_parameter_values = best_model.get_details()['runDefinition']['arguments']
print(best_model_metrics, best_model_parameter_values)
print(best_model.get_file_names())

{'Regularization Strength:': 0.8786056300135654, 'Max iterations:': 50, 'Accuracy': 0.9136926438455936} ['--C', '0.8786056300135654', '--max_iter', '50']
['azureml-logs/55_azureml-execution-tvmps_9e83af49a945e9068066263d18908a944ed9c0d9b6ce4731131a5efba4c30721_d.txt', 'azureml-logs/65_job_prep-tvmps_9e83af49a945e9068066263d18908a944ed9c0d9b6ce4731131a5efba4c30721_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_9e83af49a945e9068066263d18908a944ed9c0d9b6ce4731131a5efba4c30721_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']


In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [18]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [19]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [20]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_ouput=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_057c45c8-deb7-46c6-bff2-b8d6d565a70e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_model, fitted_model = automl_run.get_output()

In [22]:
best_automl_model.get_metrics()

{'norm_macro_recall': 0.48367248263621815,
 'recall_score_micro': 0.9169043461128664,
 'matthews_correlation': 0.5426208340057936,
 'recall_score_macro': 0.7418362413181091,
 'balanced_accuracy': 0.7418362413181091,
 'AUC_micro': 0.9809918138075099,
 'weighted_accuracy': 0.9604170684980976,
 'f1_score_macro': 0.7679080669978681,
 'accuracy': 0.9169043461128664,
 'precision_score_weighted': 0.9101950820107166,
 'precision_score_micro': 0.9169043461128664,
 'AUC_macro': 0.948627429211112,
 'precision_score_macro': 0.8046657325777445,
 'AUC_weighted': 0.9486274292111121,
 'average_precision_score_macro': 0.8273780680317246,
 'f1_score_weighted': 0.9121896457400456,
 'recall_score_weighted': 0.9169043461128664,
 'log_loss': 0.17484436583607507,
 'average_precision_score_weighted': 0.9561603142464117,
 'f1_score_micro': 0.9169043461128664,
 'average_precision_score_micro': 0.9817816234010862,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_057c45c8-deb7-46c6-bff2-b8d6d565a70e

In [24]:
print(best_automl_model.get_file_names())

['accuracy_table', 'confusion_matrix', 'explanation/0647f2f9/classes.interpret.json', 'explanation/0647f2f9/eval_data_viz.interpret.json', 'explanation/0647f2f9/expected_values.interpret.json', 'explanation/0647f2f9/features.interpret.json', 'explanation/0647f2f9/global_names/0.interpret.json', 'explanation/0647f2f9/global_rank/0.interpret.json', 'explanation/0647f2f9/global_values/0.interpret.json', 'explanation/0647f2f9/local_importance_values.interpret.json', 'explanation/0647f2f9/per_class_names/0.interpret.json', 'explanation/0647f2f9/per_class_rank/0.interpret.json', 'explanation/0647f2f9/per_class_values/0.interpret.json', 'explanation/0647f2f9/rich_metadata.interpret.json', 'explanation/0647f2f9/true_ys_viz.interpret.json', 'explanation/0647f2f9/visualization_dict.interpret.json', 'explanation/8ea99950/classes.interpret.json', 'explanation/8ea99950/expected_values.interpret.json', 'explanation/8ea99950/features.interpret.json', 'explanation/8ea99950/global_names/0.interpret.jso

In [25]:
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

